In [1]:
from pathlib import Path
import cv2, json, numpy as np
from PIL import Image
from tqdm import tqdm

import torch
torch.cuda.is_available()

True

### 路径说明
```
FS2K
├─data # FS2K数据集位置
│  └─FS2K
│      ├─photo
│      │  ├─photo1
│      │  ├─photo2
│      │  └─photo3
│      └─sketch
│          ├─sketch1
│          ├─sketch2
│          └─sketch3
├─save # 模型保存位置
└─FS2K.ipynb # 代码
```

In [2]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 定义Dataset
class DS(Dataset):
    def __init__(s, dataD, mode='train'):
        super().__init__()
        s.dataD = dataD
        s.mode = mode
        s.xtf = transforms.Compose([
            transforms.Resize((250,250)),
            transforms.ToTensor(),
        ])
        s.ytf = transforms.Compose([
            torch.tensor,
        ])
        s.data = s.read()
    
    def read(s):
        D = s.dataD
        jp = D / f'anno_{s.mode}.json'
        with jp.open('r', encoding='utf-8')as f:
            annos = json.load(f)
        return annos

    def __getitem__(s, i):
        a = s.data[i]
        imgP = s.dataD/ f"photo/{a['image_name']}.jpg"
        img = s.xtf(Image.open(imgP.as_posix()))
        colors = a['lip_color']+a['eye_color']
        attrs = list(map(int,[a['hair'],a['hair_color'],a['gender'],a['earring'],a['smile'],a['frontal_face']]))
        return img, s.ytf(colors), torch.tensor(attrs, dtype=int)

    def __len__(s):
        return len(s.data)

# 实例化Dataset
dataD = Path('./data/FS2K')
train_ds = DS(dataD)
val_ds = DS(dataD, 'test')

# 创建Dataloader
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=0)
val_dl = DataLoader(val_ds, batch_size=16, shuffle=True, num_workers=0)

### Dataset说明
每一个样本包含三个变量img, colors, attrs  
img为tensor图片  
colors为一个6元素的float类型一维数组, 前三个表示嘴唇颜色lip_color, 后三个表示眼睛颜色eye_color  
attrs为6元素的整型一位数组, 分别为hair, hair_color, gender, earring, smile, frontal_face

In [3]:
x, colors, attrs = train_ds[0]
colors, attrs

(tensor([156.9775,  82.5112,  79.0000, 118.6518,  72.2589,  69.5982]),
 tensor([0, 2, 0, 1, 1, 1]))

In [4]:
{
    "image_name": "photo1/image0110",

    "skin_patch": [163, 139],
    # a point of face region.

    "lip_color": [156.97750511247443, 82.51124744376278, 79.0],
    # the mean RGB value of lip area.

    "eye_color": [118.65178571428571, 72.25892857142857, 69.59821428571429],
    # the mean RGB value of eye area.

    "hair": 0,
    # 0: with hair, 1: without hair.

    "hair_color": 2,
    # 0: brown, 1: black, 2: red, 3: no-hair, 4: golden.

    "gender": 0,
    # 0: male, 1: female.

    "earring": 1,
    # 0: with earring, 1: without earring.

    "smile": 1,
    # 0: with smile, 1: without smile.

    "frontal_face": 1,
    # 0: head rotates within 30 degrees, 1: > 30 degrees

    "style": 0
    # Style = one of {0, 1, 2}, please refer to the sketch samples.
}

{'image_name': 'photo1/image0110',
 'skin_patch': [163, 139],
 'lip_color': [156.97750511247443, 82.51124744376278, 79.0],
 'eye_color': [118.65178571428571, 72.25892857142857, 69.59821428571429],
 'hair': 0,
 'hair_color': 2,
 'gender': 0,
 'earring': 1,
 'smile': 1,
 'frontal_face': 1,
 'style': 0}

In [40]:
import torch
import torch.nn as nn

class _Hswish(nn.Module):
    def __init__(self, inplace=True):
        super(_Hswish, self).__init__()
        self.relu6 = nn.ReLU6(inplace)

    def forward(self, x):
        return x * self.relu6(x + 3.) / 6.


class _Hsigmoid(nn.Module):
    def __init__(self, inplace=True):
        super(_Hsigmoid, self).__init__()
        self.relu6 = nn.ReLU6(inplace)

    def forward(self, x):
        return self.relu6(x + 3.) / 6.


class _ConvBNHswish(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0,
                 dilation=1, groups=1, norm_layer=nn.BatchNorm2d, **kwargs):
        super(_ConvBNHswish, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias=False)
        self.bn = norm_layer(out_channels)
        self.act = _Hswish(True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.act(x)
        return x


class SEModule(nn.Module):
    def __init__(self, in_channels, reduction=4):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            _Hsigmoid(True)
        )

    def forward(self, x):
        n, c, _, _ = x.size()
        out = self.avg_pool(x).view(n, c)
        out = self.fc(out).view(n, c, 1, 1)
        return x * out.expand_as(x)


class Identity(nn.Module):
    def __init__(self, in_channels):
        super(Identity, self).__init__()

    def forward(self, x):
        return x


class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, exp_size, kernel_size, stride, dilation=1, se=False, nl='RE',
                 norm_layer=nn.BatchNorm2d, **kwargs):
        super(Bottleneck, self).__init__()
        assert stride in [1, 2]
        self.use_res_connect = stride == 1 and in_channels == out_channels
        if nl == 'HS':
            act = _Hswish
        else:
            act = nn.ReLU
        if se:
            SELayer = SEModule
        else:
            SELayer = Identity

        self.conv = nn.Sequential(
            # pw
            nn.Conv2d(in_channels, exp_size, 1, bias=False),
            norm_layer(exp_size),
            act(True),
            # dw
            nn.Conv2d(exp_size, exp_size, kernel_size, stride, (kernel_size - 1) // 2 * dilation,
                      dilation, groups=exp_size, bias=False),
            norm_layer(exp_size),
            SELayer(exp_size),
            act(True),
            # pw-linear
            nn.Conv2d(exp_size, out_channels, 1, bias=False),
            norm_layer(out_channels)
        )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV3(nn.Module):
    def __init__(self, nclass=1000, mode='small', width_mult=1.0, dilated=False, norm_layer=nn.BatchNorm2d):
        super(MobileNetV3, self).__init__()
        self.mode = mode
        if mode == 'large':
            layer1_setting = [
                # k, exp_size, c, se, nl, s
                [3, 16, 16, False, 'RE', 1],
                [3, 64, 24, False, 'RE', 2],
                [3, 72, 24, False, 'RE', 1], ]
            layer2_setting = [
                [5, 72, 40, True, 'RE', 2],
                [5, 120, 40, True, 'RE', 1],
                [5, 120, 40, True, 'RE', 1], ]
            layer3_setting = [
                [3, 240, 80, False, 'HS', 2],
                [3, 200, 80, False, 'HS', 1],
                [3, 184, 80, False, 'HS', 1],
                [3, 184, 80, False, 'HS', 1],
                [3, 480, 112, True, 'HS', 1],
                [3, 672, 112, True, 'HS', 1],
                [5, 672, 112, True, 'HS', 1], ]
            layer4_setting = [
                [5, 672, 160, True, 'HS', 2],
                [5, 960, 160, True, 'HS', 1], ]
        elif mode == 'small':
            layer1_setting = [
                # k, exp_size, c, se, nl, s
                [3, 16, 16, True, 'RE', 2], ]
            layer2_setting = [
                [3, 72, 24, False, 'RE', 2],
                [3, 88, 24, False, 'RE', 1], ]
            layer3_setting = [
                [5, 96, 40, True, 'HS', 2],
                [5, 240, 40, True, 'HS', 1],
                [5, 240, 40, True, 'HS', 1],
                [5, 120, 48, True, 'HS', 1],
                [5, 144, 48, True, 'HS', 1], ]
            layer4_setting = [
                [5, 288, 96, True, 'HS', 2],
                [5, 576, 96, True, 'HS', 1],
                [5, 576, 96, True, 'HS', 1], ]
        else:
            raise ValueError('Unknown mode.')

        # building first layer
        self.in_channels = int(16 * width_mult) if width_mult > 1.0 else 16
        self.conv1 = _ConvBNHswish(3, self.in_channels, 3, 2, 1, norm_layer=norm_layer)

        # building bottleneck blocks
        self.layer1 = self._make_layer(Bottleneck, layer1_setting,
                                       width_mult, norm_layer=norm_layer)
        self.layer2 = self._make_layer(Bottleneck, layer2_setting,
                                       width_mult, norm_layer=norm_layer)
        self.layer3 = self._make_layer(Bottleneck, layer3_setting,
                                       width_mult, norm_layer=norm_layer)
        if dilated:
            self.layer4 = self._make_layer(Bottleneck, layer4_setting,
                                           width_mult, dilation=2, norm_layer=norm_layer)
        else:
            self.layer4 = self._make_layer(Bottleneck, layer4_setting,
                                           width_mult, norm_layer=norm_layer)

        # building last several layers
        classifier = list()
        if mode == 'large':
            last_bneck_channels = int(960 * width_mult) if width_mult > 1.0 else 960
            self.layer5 = _ConvBNHswish(self.in_channels, last_bneck_channels, 1, norm_layer=norm_layer)
            classifier.append(nn.AdaptiveAvgPool2d(1))
            classifier.append(nn.Conv2d(last_bneck_channels, 1280, 1))
            classifier.append(_Hswish(True))
#             classifier.append(nn.Conv2d(1280, nclass, 1))
        elif mode == 'small':
            last_bneck_channels = int(576 * width_mult) if width_mult > 1.0 else 576
            self.layer5 = _ConvBNHswish(self.in_channels, last_bneck_channels, 1, norm_layer=norm_layer)
            classifier.append(SEModule(last_bneck_channels))
            classifier.append(nn.AdaptiveAvgPool2d(1))
            classifier.append(nn.Conv2d(last_bneck_channels, 1280, 1))
            classifier.append(_Hswish(True))
#             classifier.append(nn.Conv2d(1280, nclass, 1))
        else:
            raise ValueError('Unknown mode.')
#         self.classifier = nn.Sequential(*classifier)

        self.flat1280 = nn.Sequential(*classifier)
        n = 1280
        self.lip_color = nn.Conv2d(n, 3, 1)
        self.eye_color = nn.Conv2d(n, 3, 1)
        self.hair = nn.Conv2d(n, 2, 1)
        self.hair_color = nn.Conv2d(n, 5, 1)
        self.gender = nn.Conv2d(n, 2, 1)
        self.earring = nn.Conv2d(n, 2, 1)
        self.smile = nn.Conv2d(n, 2, 1)
        self.frontal_face = nn.Conv2d(n, 2, 1)

        self._init_weights()

    def _make_layer(self, block, block_setting, width_mult, dilation=1, norm_layer=nn.BatchNorm2d):
        layers = list()
        for k, exp_size, c, se, nl, s in block_setting:
            out_channels = int(c * width_mult)
            stride = s if (dilation == 1) else 1
            exp_channels = int(exp_size * width_mult)
            layers.append(block(self.in_channels, out_channels, exp_channels, k, stride, dilation, se, nl, norm_layer))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
#         x = self.classifier(x)
        flat1280 = self.flat1280(x)
#         x = x.view(x.size(0), x.size(1))

        lip_color = self.lip_color(flat1280)
        lip_color = lip_color.view(lip_color.size(0), lip_color.size(1))
        eye_color = self.eye_color(flat1280)
        eye_color = eye_color.view(eye_color.size(0), eye_color.size(1))
        hair = self.hair(flat1280)
        hair = hair.view(hair.size(0), hair.size(1))
        hair_color = self.hair_color(flat1280)
        hair_color = hair_color.view(hair_color.size(0), hair_color.size(1))
        gender = self.gender(flat1280)
        gender = gender.view(gender.size(0), gender.size(1))
        earring = self.earring(flat1280)
        earring = earring.view(earring.size(0), earring.size(1))
        smile = self.smile(flat1280)
        smile = smile.view(smile.size(0), smile.size(1))
        frontal_face = self.frontal_face(flat1280)
        frontal_face = frontal_face.view(frontal_face.size(0), frontal_face.size(1))
        
        return [lip_color, eye_color, hair, hair_color, gender, earring, smile, frontal_face]

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

def get_mobilenet_v3(mode='small', width_mult=1.0, pretrained=False, root='~/.torch/models', **kwargs):
    model = MobileNetV3(mode=mode, width_mult=width_mult, **kwargs)
    if pretrained:
        raise ValueError("Not support pretrained")
    return model


def mobilenet_v3_large_1_0(**kwargs):
    return get_mobilenet_v3('large', 1.0, **kwargs)


def mobilenet_v3_small_1_0(**kwargs):
    return get_mobilenet_v3('small', 1.0, **kwargs)


class Loss(torch.nn.Module):
    def __init__(s):
        super().__init__()
        s.MSE = torch.nn.MSELoss()
        s.CE = torch.nn.CrossEntropyLoss()

    def forward(s, preds, colors_b, attrs_b):
#         y = a['lip_color']+a['eye_color']+[a['hair'],a['hair_color'],a['gender'],a['earring'],a['smile'],a['frontal_face']]
        lip_color, eye_color, hair, hair_color, gender, earring, smile, frontal_face = preds
        lpc = s.MSE(lip_color, colors_b[:,:3])
        lc = s.MSE(eye_color, colors_b[:,3:])
        h = s.CE(hair, attrs_b[:, 0])
        hc = s.CE(hair_color, attrs_b[:, 1])
        g = s.CE(gender, attrs_b[:, 2])
        e = s.CE(earring, attrs_b[:, 3])
        sm = s.CE(smile, attrs_b[:, 4])
        f = s.CE(frontal_face, attrs_b[:, 5])
        loss = lpc+lc+h+hc+g+e+sm+f
        
        return loss


### 模型和损失函数说明
模型使用Mobilenetv3的Small版本， 在将最后的输出层更改为8个并行的1x1卷积层，对2个颜色属性进行回归，对6个整型属性进行分类  
回归损失使用MSE，分类损失使用交叉熵损失

In [7]:
def save(savePath, m, epoch, acc):
    d = {
        'param': m.state_dict(),
        'epoch': epoch,
        'acc': acc,
    }
    if isinstance(savePath, Path):
        savePath = savePath.as_posix()
    torch.save(d, savePath)
    print('checkpoint saved as', savePath)

def load(loadPath):
    if isinstance(loadPath, Path):
        loadPath = loadPath.as_posix()
    d = torch.load(loadPath)
    m = Model()
    m.load_state_dict(d['param'])
    print('checkpoint loaded from', loadPath)
    e, acc = d['epoch'], d['acc']
    print('epoch:', e, 'acc:', acc)
    return m, d['epoch'], d['acc']

def toCpu(path):
    path = Path(path)
    m, e, acc = load(path)
    m.to(torch.device('cpu'))
    save(path.parents[0]/f'{path.stem}_cpu.ckpt')

In [34]:
def val(em, param, val_dl, d):
    l=Loss()
    vn = len(val_dl.dataset)
    em.load_state_dict(param)
    hair_cnt = 0
    hair_color_cnt = 0
    gender_cnt = 0
    earring_cnt = 0
    smile_cnt = 0
    frontal_face_cnt = 0
    L = 0
    with torch.no_grad():
        for i, (xs, colors_b, attrs_b) in enumerate(val_dl):
            xs, colors_b, attrs_b = xs.to(d), colors_b.to(d), attrs_b.to(d)
            outs = em(xs)
            lip_color, eye_color, hair, hair_color, gender, earring, smile, frontal_face = outs
            L += l(outs, colors_b, attrs_b.long()).item()
            hair_ = torch.max(hair, 1)[1]
            hair_cnt += torch.sum(hair_ == attrs_b[:,0])
            
            hair_color_ = torch.max(hair_color, 1)[1]
            hair_color_cnt += torch.sum(hair_color_ == attrs_b[:,1])
            
            gender_ = torch.max(gender, 1)[1]
            gender_cnt += torch.sum(gender_ == attrs_b[:,2])
            
            earring_ = torch.max(earring, 1)[1]
            earring_cnt += torch.sum(earring_ == attrs_b[:,3])
            
            smile_ = torch.max(smile, 1)[1]
            smile_cnt += torch.sum(smile_ == attrs_b[:,4])
            
            frontal_face_ = torch.max(frontal_face, 1)[1]
            frontal_face_cnt += torch.sum(frontal_face_ == attrs_b[:,5])
    acc = (hair_cnt+hair_color_cnt+gender_cnt+earring_cnt+smile_cnt+frontal_face_cnt)/6/vn
    print(f'validated on {vn} samples| mean acc:{acc*100:.4f}%')
    print(f'hair_cnt:{hair_cnt/vn}|hair_color_cnt:{hair_color_cnt/vn}|gender_cnt:{gender_cnt/vn}')
    print(f'earring_cnt:{earring_cnt/vn}|smile_cnt:{smile_cnt/vn}|frontal_face_cnt:{frontal_face_cnt/vn}')
    print(f'loss:{L/vn:.4f}')
    
    return acc

def train(m,
          d,
          train_dl,
          val_dl,
          saveDir=Path('save'),
          resumePath=None,
          lr=0.001,
          e=50,
          s=10
         ):
    saveDir.mkdir(exist_ok=1)
    startEp = -1
    b = 0
    try:
        m, startEp, b = load(saveDir/'best.ckpt')
    except Exception as err:
        print(err)

    if resumePath is not None:
        m, startEp, b = load(resumePath)

    m.to(d).train()
    em = Model().to(d).eval()
    
    l=Loss()
    o=torch.optim.SGD(m.parameters(), lr=lr, momentum=0.9)
    
    saveDir.mkdir(exist_ok=1)
    tn = len(train_dl.dataset)
    t = tqdm(range(startEp+1, e))
#     t = range(startEp+1, e)
    for ep in t:
        L = 0
        for i, (xs, colors_b, attrs_b) in enumerate(train_dl):
            xs, colors_b, attrs_b = xs.to(d), colors_b.to(d), attrs_b.to(d)
            o.zero_grad()
            outs = m(xs)
            loss = l(outs, colors_b, attrs_b.long())
            loss.backward()
            o.step()

            L += loss.item()
        t.set_description(f'ep:{ep}| L:{L/tn:.6f}')
        if (ep+1)%s != 0: continue

        acc = val(em, m.state_dict(), val_dl, d)
        save(saveDir/f'{ep:05d}_{acc:.4f}.ckpt', m, ep, acc)
        if b < acc:
            b = acc
            save(saveDir/'best.ckpt', m, ep, acc)
        print(f'E:{ep}| L:{L/tn:.6f}')

In [33]:
d = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(d)
# d = torch.device('cpu')
Model = MobileNetV3
m = Model().to(d)
train(m,
      d,
      train_dl,
      val_dl,
      saveDir=Path('save'),
#       resumePath=Path('save2/03999_0.8062.ckpt'),
      lr=0.0001,
      e=4000,
      s=80)

cuda:0
checkpoint loaded from save/best.ckpt
epoch: 19 acc: tensor(0.6475, device='cuda:0')


ep:79| L:25.774263:   2%|▊                                                        | 60/3980 [08:45<10:51:17,  9.97s/it]

validated on 1046 samples| acc:68.8496%
loss:45.7624
checkpoint saved as save/00079_0.6885.ckpt
checkpoint saved as save/best.ckpt
E:79| L:25.774263


ep:159| L:14.791611:   3%|█▉                                                      | 139/3980 [20:26<9:20:25,  8.75s/it]

validated on 1046 samples| acc:73.6456%
loss:34.0878


...

checkpoint saved as save/03839_0.7541.ckpt
E:3839| L:0.412955


ep:3919| L:0.356495:  98%|█████████████████████████████████████████████████████▉ | 3900/3980 [9:33:12<13:15,  9.95s/it]

validated on 1046 samples| acc:75.2071%
loss:31.4749
checkpoint saved as save/03919_0.7521.ckpt
E:3919| L:0.356495


ep:3999| L:0.282235: 100%|███████████████████████████████████████████████████████| 3980/3980 [9:44:59<00:00,  8.82s/it]

validated on 1046 samples| acc:75.1434%
loss:31.6003
checkpoint saved as save/03999_0.7514.ckpt
E:3999| L:0.282235


In [37]:
em, epo, acc = load('save/best.ckpt')
val(Model().to(d).eval(), em.state_dict(), val_dl, d)

checkpoint loaded from save/best.ckpt
epoch: 3759 acc: tensor(0.7583, device='cuda:0')
validated on 1046 samples| mean acc:75.8286%
hair_cnt:0.944550633430481|hair_color_cnt:0.5009559988975525|gender_cnt:0.8355640769004822
earring_cnt:0.7877628803253174|smile_cnt:0.6653919816017151|frontal_face_cnt:0.8154875636100769
loss:31.4907


tensor(0.7583, device='cuda:0')

In [41]:
d = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(d)
# d = torch.device('cpu')
Model = MobileNetV3
m = Model().to(d)
print(m.mode)
train(m,
      d,
      train_dl,
      val_dl,
      saveDir=Path('save_small'),
      lr=0.0001,
      e=4000,
      s=80)

cuda:0
small
[Errno 2] No such file or directory: 'save_small/best.ckpt'


ep:79| L:22.012985:   2%|█▏                                                        | 79/4000 [07:16<5:59:21,  5.50s/it]

validated on 1046 samples| mean acc:73.4385%
hair_cnt:0.9502868056297302|hair_color_cnt:0.44455066323280334|gender_cnt:0.7227533459663391
earring_cnt:0.8097514510154724|smile_cnt:0.6453154683113098|frontal_face_cnt:0.8336520195007324
loss:34.8414
checkpoint saved as save_small/00079_0.7344.ckpt


ep:79| L:22.012985:   2%|█▏                                                       | 80/4000 [07:33<11:30:19, 10.57s/it]

checkpoint saved as save_small/best.ckpt
E:79| L:22.012985


...

validated on 1046 samples| mean acc:75.6214%
hair_cnt:0.9502868056297302|hair_color_cnt:0.5066921710968018|gender_cnt:0.8087953925132751
earring_cnt:0.8116634488105774|smile_cnt:0.6539196968078613|frontal_face_cnt:0.8059273362159729
loss:31.8718
checkpoint saved as save_small/03919_0.7562.ckpt
E:3919| L:1.232528


ep:3999| L:0.462838: 100%|██████████████████████████████████████████████████████▉| 3999/4000 [6:05:49<00:05,  5.41s/it]

validated on 1046 samples| mean acc:75.6692%
hair_cnt:0.9483747482299805|hair_color_cnt:0.49713191390037537|gender_cnt:0.8107074499130249
earring_cnt:0.8011472225189209|smile_cnt:0.6673040390014648|frontal_face_cnt:0.8154875636100769
loss:31.6671


ep:3999| L:0.462838: 100%|███████████████████████████████████████████████████████| 4000/4000 [6:05:52<00:00,  5.49s/it]

checkpoint saved as save_small/03999_0.7567.ckpt
E:3999| L:0.462838


In [43]:
em, epo, acc = load('save_small/best.ckpt')
val(Model().to(d).eval(), em.state_dict(), val_dl, d)

checkpoint loaded from save_small/best.ckpt
epoch: 3759 acc: tensor(0.7632, device='cuda:0')
validated on 1046 samples| mean acc:76.3225%
hair_cnt:0.9483747482299805|hair_color_cnt:0.5172083973884583|gender_cnt:0.8126195073127747
earring_cnt:0.8212236762046814|smile_cnt:0.6682600378990173|frontal_face_cnt:0.8116634488105774
loss:32.5743


tensor(0.7632, device='cuda:0')